## Calculate terminal velocites and travel distances

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipdb
from textwrap import dedent
%matplotlib inline

import os
# Prepare working directories.  Notebook should be launched from within src directory.
src_dir = '../src'
data_dir = '../data'
plots_dir = '../plots'
os.chdir(src_dir)

from travdist import density, fall_velocity, particle

In [2]:
# Load the data
xmt = pd.read_csv(os.path.join(data_dir, 'xmt.csv'))
xmt = xmt.copy().iloc[1:len(xmt):50, :]  # Take 2% of the dataset to speed calculation
#xmt = xmt.copy().iloc[1:len(xmt):10, :]  # Take 10% of the dataset to speed calculation

In [3]:
def add_fall_velocity_cols(df):
    """Add columns with terminal velocities from different Ganser schemes.
    ganser_rhoXMT, ganser_sphXMT, ganser_sphXMT_conv, ganser_rhoXMT_sphXMT.
    """
    df['white_theory'] = df.index.map(lambda x: fall_velocity.white(1e-6*df.loc[x, 'MicronsDiameter'],
                                                                    density=2300))
    df['ganser_theory'] = df.index.map(lambda x: fall_velocity.ganser(1e-6*df.loc[x, 'MicronsDiameter'],
                                                                   sphericity=0.7,
                                                                   density=df.loc[x, 'DensityBP2003']))
    df['ganser_rhoXMT'] = df.index.map(lambda x: fall_velocity.ganser(1e-6*df.loc[x, 'MicronsDiameter'],
                                                                   sphericity=0.7,
                                                                   density=df.loc[x, 'EffectiveDensity']))
    df['ganser_sphXMT'] = df.index.map(lambda x: fall_velocity.ganser(1e-6*df.loc[x, 'MicronsDiameter'],
                                                                   sphericity=df.loc[x, 'Sphericity'],
                                                                   density=df.loc[x, 'DensityBP2003']))
    df['ganser_sphXMT_conv'] = df.index.map(lambda x: fall_velocity.ganser(1e-6*df.loc[x, 'MicronsDiameter'],
                                                                   sphericity=df.loc[x, 'ConvexHullSphericity'],
                                                                   density=df.loc[x, 'DensityBP2003']))
    df['ganser_fullXMT'] = df.index.map(lambda x: fall_velocity.ganser(1e-6*df.loc[x, 'MicronsDiameter'],
                                                                   sphericity=df.loc[x, 'Sphericity'],
                                                                   density=df.loc[x, 'EffectiveDensity']))



def add_travel_distance_cols(df):
    """Add column with travel distance."""
    # Extract useful data into lists
    diameters = df['MicronsDiameter'].tolist()
    densities_bp2003 = df['DensityBP2003'].tolist()
    densities_XMT = df['EffectiveDensity'].tolist()
    sphericities_XMT = df['Sphericity'].tolist()
    sphericities_XMT_conv = df['ConvexHullSphericity'].tolist()
    
    # Define which lists apply to which method
    data_cols = {'white_theory': dict(density=[2300]*len(df), sphericity=[0.7]*len(df)),
                 'ganser_theory': dict(density=densities_bp2003, sphericity=[0.7]*len(df)),
                 'ganser_rhoXMT': dict(density=densities_XMT, sphericity=[0.7]*len(df)),
                 'ganser_sphXMT': dict(density=densities_bp2003, sphericity=sphericities_XMT),
                 'ganser_sphXMT_conv': dict(density=densities_bp2003, sphericity=sphericities_XMT_conv),
                 'ganser_fullXMT': dict(density=densities_XMT, sphericity=sphericities_XMT)}
    
    # Calculate distances for each method
    for method, inputs in data_cols.items():
        print('Method: {}'.format(method))
        distances = []
        for i in range(len(diameters)):
            p = particle.Particle(1e-6*diameters[i], particle_density=inputs['density'][i],
                                  sphericity=inputs['sphericity'][i])
            distances.append(p.calculate_distance(velocity_function='ganser'))
        
        # Add to dataframe
        distance_column_name = 'dist_' + method
        df[distance_column_name] = distances

In [4]:
# Calculate fall velocity at sea level
add_fall_velocity_cols(xmt)

# Print some stats
fall_velocity_columns = ['white_theory', 'ganser_theory', 'ganser_rhoXMT',
                         'ganser_sphXMT', 'ganser_sphXMT_conv', 'ganser_fullXMT']
xmt[fall_velocity_columns].describe()

,white_theory,ganser_theory,ganser_rhoXMT,ganser_sphXMT,ganser_sphXMT_conv,ganser_fullXMT
count,9.600000e+01,96.000000,95.000000,95.000000,95.000000,95.000000
mean,1.399238e-05,0.281958,0.254833,0.228170,0.309792,0.212411
std,1.659731e-05,0.198615,0.229911,0.154481,0.226931,0.188599
min,3.634804e-07,0.018736,0.008929,0.015504,0.017454,0.007416
25%,3.998389e-06,0.141586,0.091754,0.117457,0.151664,0.070721
50%,6.809113e-06,0.209555,0.151230,0.167726,0.228567,0.121778
75%,1.658406e-05,0.376951,0.351283,0.312007,0.422434,0.291879
max,8.906138e-05,0.903735,0.984526,0.663661,0.983053,0.795778


In [5]:
# Calculate travel distances
add_travel_distance_cols(xmt)

# Print some stats
travel_distance_columns = ['dist_white_theory', 'dist_ganser_theory', 'dist_ganser_rhoXMT',
                           'dist_ganser_sphXMT', 'dist_ganser_sphXMT_conv', 'dist_ganser_fullXMT']
xmt[travel_distance_columns].describe()

Method: ganser_theory
Method: ganser_fullXMT
Method: ganser_sphXMT_conv
Method: ganser_rhoXMT
Method: white_theory
Method: ganser_sphXMT


,dist_white_theory,dist_ganser_theory,dist_ganser_rhoXMT,dist_ganser_sphXMT,dist_ganser_sphXMT_conv,dist_ganser_fullXMT
count,96.000000,96.000000,95.000000,95.000000,95.000000,95.000000
mean,440.153596,586.206601,952.549498,692.638511,561.737110,1111.270314
std,659.971507,760.861262,1577.946457,856.838960,769.109024,1781.661266
min,54.301763,91.705256,84.224929,124.781718,85.090026,103.739536
25%,149.046823,227.404579,244.538282,273.210123,204.300434,291.482062
50%,289.028363,417.008781,581.125389,517.806234,384.686763,716.398149
75%,448.606279,623.436167,967.117132,745.683169,583.954041,1249.835116
max,4207.126360,4819.328312,10125.532330,5814.361140,5169.672735,12178.668410


In [6]:
xmt.to_csv(os.path.join(data_dir, 'xmt_with_fall.csv'), index=False)